In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from cleandata import add_impact_score, get_data, plot_grouped_impact_scores, remove_outliers

In [ ]:
df = get_data()
df = add_impact_score(df)
df = remove_outliers(df)

In [ ]:
plot_grouped_impact_scores(df)

In [ ]:
# First, let's reload the data to see what we're working with
df = get_data()
df = add_impact_score(df)
df = remove_outliers(df)

# Check the unique values in graduation year column
# print("Unique graduation years:")
# print(df['años de graduación'].value_counts())
# print(f"\nTotal rows before filtering: {len(df)}")

# Check how many rows would be removed
# rows_to_remove = df[(df['años de graduación'] == 2022) | (df['años de graduación'] == 'Sin dato')]
# print(f"Rows that would be removed: {len(rows_to_remove)}")

# Apply the filter
df_filtered = df[~((df['años de graduación'] == '2022') |
                   (df['años de graduación'] == 'Sin dato'))]
print(f"Rows remaining after filtering: {len(df_filtered)}")

# Plot with the filtered data
if len(df_filtered) > 0:
    plot_grouped_impact_scores(df_filtered)
else:
    print("No data remaining after filtering. Consider adjusting the filter criteria.")

In [ ]:
print(1, len(pd.read_csv('80QS.csv')))
print(2, len(df_filtered))

In [ ]:
X, Y = np.unique(df_filtered['años de graduación'], return_counts=True)
plt.figure(figsize=(10, 6))
sns.barplot(x=X, y=Y, palette='viridis')
plt.title('Graduation Year Distribution')
plt.xlabel('Graduation Year')
plt.ylabel('Number of Students')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('graduation_year_distribution.png')
plt.show()

In [ ]:
# Regression models
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Prepare the data for regression
grouped = df_filtered.groupby('años de graduación')[
    'impact_score'].sum().reset_index()
X = grouped['años de graduación'].values.reshape(-1, 1)
y = grouped['impact_score'].values
# Convert years to numeric values
X = X.astype(float)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
# Create and fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# Make predictions
y_pred = model.predict(X_test)
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')
# Plot the regression results
plt.figure(figsize=(10, 6))
# Plot actual data as bars
plt.bar(X.flatten(), y, color='blue', alpha=0.5, label='Actual')
# Plot regression line
X_range = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
y_range_pred = model.predict(X_range)
plt.plot(X_range.flatten(), y_range_pred, color='red',
         linewidth=2, label='Regression Line')
# Plot test predictions
plt.scatter(X_test.flatten(), y_pred, color='orange',
            s=50, label='Test Predictions', zorder=5)
plt.title('Impact Score vs Graduation Year')
plt.xlabel('Graduation Year')
plt.ylabel('Impact Score')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
# plt.savefig('impact_score_vs_graduation_year.png')
plt.show()

In [ ]:
# ARIMA model
from statsmodels.tsa.arima.model import ARIMA
# Prepare the data for ARIMA
grouped = df_filtered.groupby('años de graduación')[
    'impact_score'].sum().reset_index()
# Convert years to datetime format
grouped['años de graduación'] = pd.to_datetime(
    grouped['años de graduación'], format='%Y', errors='coerce')
# Set the index to the graduation year
grouped.set_index('años de graduación', inplace=True)
# Ensure the index is sorted
grouped.sort_index(inplace=True)
print("Grouped Data for ARIMA:")
print(grouped.head())
# Fit the ARIMA model
model = ARIMA(grouped['impact_score'], order=(0, 1, 5))
model_fit = model.fit()
# Make predictions
# for each year in the next 5 years
forecast = model_fit.forecast(steps=5)
# Print the forecasted values
print("ARIMA Forecast for the next 5 years:")
print(forecast)
# Create future dates for forecasting
last_date = grouped.index[-1]
future_dates = pd.date_range(
    start=last_date + pd.DateOffset(years=1), periods=5, freq='YE')

plt.figure(figsize=(12, 6))
# Plot historical data using the index (which is now the datetime)
plt.plot(grouped.index, grouped['impact_score'],
         label='Historical Data', color='blue', marker='o')

# Plot forecast
plt.plot(future_dates, forecast,
         label='ARIMA Forecast', color='red', marker='s', linestyle='--')

plt.title('ARIMA Forecast of Impact Score')
plt.xlabel('Graduation Year')
plt.ylabel('Impact Score')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
# plt.savefig('arima_forecast_impact_score.png')
plt.show()

# Print the forecast with corresponding years
print("\nDetailed Forecast:")
for i, (date, value) in enumerate(zip(future_dates, forecast)):
    print(f"Year {date.year}: {value:.2f}")

In [ ]:
# Support Vector Regression (SVR)
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# Prepare the data for SVR - use year values instead of datetime
grouped_reset = grouped.reset_index()
grouped_reset['year'] = grouped_reset['años de graduación'].dt.year

X = grouped_reset['year'].values.reshape(-1, 1)
y = grouped_reset['impact_score'].values

# Scale the features for better SVR performance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(
#     X_scaled, y, test_size=0.2, random_state=42)
len_data = len(X_scaled)
X_train, X_test, y_train, y_test = X_scaled[:int(
    len_data*0.9)], X_scaled[int(len_data*0.9):], y[:int(len_data*0.9)], y[int(len_data*0.9):]

# Create and fit the SVR model
svr_model = SVR(kernel='rbf', C=10000000, gamma='scale', epsilon=0.1)
svr_model.fit(X_train, y_train)

# Make predictions
y_svr_pred = svr_model.predict(X_test)

# Evaluate the SVR model
mse_svr = mean_squared_error(y_test, y_svr_pred)
r2_svr = r2_score(y_test, y_svr_pred)
print(f'SVR Mean Squared Error: {mse_svr}')
print(f'SVR R-squared: {r2_svr}')

# Plot the SVR results
plt.figure(figsize=(10, 6))

# Plot actual data as bars using year values
years = grouped_reset['year'].values
plt.bar(years, y, color='blue', alpha=0.5, label='Actual')

# Get corresponding years for test predictions
X_test_years = scaler.inverse_transform(X_test)
plt.scatter(X_test_years.flatten(), y_svr_pred, color='orange',
            s=50, label='SVR Predictions', zorder=5)

# Plot SVR line
X_range = np.linspace(years.min(), years.max(), 100).reshape(-1, 1)
X_range_scaled = scaler.transform(X_range)
y_svr_range_pred = svr_model.predict(X_range_scaled)
plt.plot(X_range.flatten(), y_svr_range_pred,
         color='green', linewidth=2, label='SVR Line')

plt.title('SVR Impact Score vs Graduation Year')
plt.xlabel('Graduation Year')
plt.ylabel('Impact Score')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Predict 5 more years into the future
future_years = np.array([2022, 2023, 2024, 2025, 2026,
                        2027, 2028, 2029, 2030]).reshape(-1, 1)
future_years_scaled = scaler.transform(future_years)
future_predictions = svr_model.predict(future_years_scaled)

print("\nSVR Forecast for the next 5 years:")
for year, prediction in zip(future_years.flatten(), future_predictions):
    print(f"Year {year}: {prediction:.2f}")

# Plot with future predictions
plt.figure(figsize=(12, 6))
plt.bar(years, y, color='blue', alpha=0.5, label='Historical Data')
plt.scatter(future_years.flatten(), future_predictions,
            color='red', s=100, label='Future Predictions', marker='s')
plt.plot(X_range.flatten(), y_svr_range_pred,
         color='green', linewidth=2, label='SVR Line')

plt.title('SVR Impact Score Forecast')
plt.xlabel('Graduation Year')
plt.ylabel('Impact Score')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Random Forest Regression
from sklearn.ensemble import RandomForestRegressor
# Create and fit the Random Forest model
rf_model = RandomForestRegressor(n_estimators=1000)
rf_model.fit(X_train, y_train)
# Make predictions
y_rf_pred = rf_model.predict(X_test)
# Evaluate the Random Forest model
mse_rf = mean_squared_error(y_test, y_rf_pred)
r2_rf = r2_score(y_test, y_rf_pred)
print(f'Random Forest Mean Squared Error: {mse_rf}')
print(f'Random Forest R-squared: {r2_rf}')
# Plot the Random Forest results
plt.figure(figsize=(10, 6))
# Plot actual data as bars using year values
plt.bar(years, y, color='blue', alpha=0.5, label='Actual')
# Get corresponding years for test predictions
X_test_years = scaler.inverse_transform(X_test)
plt.scatter(X_test_years.flatten(), y_rf_pred, color='orange',
            s=50, label='RF Predictions', zorder=5)
# Plot Random Forest line
X_range = np.linspace(years.min(), years.max(), 100).reshape(-1, 1)
X_range_scaled = scaler.transform(X_range)
y_rf_range_pred = rf_model.predict(X_range_scaled)
plt.plot(X_range.flatten(), y_rf_range_pred,
         color='purple', linewidth=2, label='RF Line')
plt.title('Random Forest Impact Score vs Graduation Year')
plt.xlabel('Graduation Year')
plt.ylabel('Impact Score')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
# Compare models
models = {
    'Linear Regression': (mse, r2),
    'SVR': (mse_svr, r2_svr),
    'Random Forest': (mse_rf, r2_rf)
}
print("\nModel Comparison:")
for model_name, (mse_value, r2_value) in models.items():
    print(f"{model_name} - Mean Squared Error: {mse_value:.2f}, R-squared: {r2_value:.2f}")

In [ ]:
# Deep Learning - Improved CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Set random seed for reproducibility
tf.random.set_seed(42)

# Prepare the data for CNN with better preprocessing
# Use the scaled data from SVR
X_cnn = X_scaled.reshape(-1, 1, 1)  # Reshape for CNN (samples, timesteps, features)
y_cnn = y

# Use the same train/test split as other models for fair comparison
X_cnn_train = X_cnn[:len_data//10*9]
X_cnn_test = X_cnn[len_data//10*9:]
y_cnn_train = y_cnn[:len_data//10*9]
y_cnn_test = y_cnn[len_data//10*9:]

# Create an improved CNN model
cnn_model = Sequential([
    Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(1, 1)),
    Dropout(0.3),
    Conv1D(filters=64, kernel_size=1, activation='relu'),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer
])

# Compile with better parameters
cnn_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# Add early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=50,
    restore_best_weights=True
)

# Fit the model with validation split
history = cnn_model.fit(
    X_cnn_train, y_cnn_train,
    epochs=1000,
    batch_size=8,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

# Make predictions
y_cnn_pred = cnn_model.predict(X_cnn_test).flatten()

# Evaluate the CNN model
mse_cnn = mean_squared_error(y_cnn_test, y_cnn_pred)
r2_cnn = r2_score(y_cnn_test, y_cnn_pred)
print(f'Improved CNN Mean Squared Error: {mse_cnn}')
print(f'Improved CNN R-squared: {r2_cnn}')

# Plot the CNN results
plt.figure(figsize=(12, 8))

# Plot actual data as bars
plt.bar(years, y, color='blue', alpha=0.5, label='Actual')

# Get corresponding years for test predictions
X_cnn_test_years = scaler.inverse_transform(X_cnn_test.reshape(-1, 1))
plt.scatter(X_cnn_test_years.flatten(), y_cnn_pred, color='orange',
            s=100, label='CNN Predictions', zorder=5)

# Plot CNN line for the full range
X_range_cnn = np.linspace(years.min(), years.max(), 100).reshape(-1, 1)
X_range_cnn_scaled = scaler.transform(X_range_cnn).reshape(-1, 1, 1)
y_cnn_range_pred = cnn_model.predict(X_range_cnn_scaled).flatten()
plt.plot(X_range_cnn.flatten(), y_cnn_range_pred,
         color='cyan', linewidth=2, label='CNN Line')

plt.title('Improved CNN Impact Score vs Graduation Year')
plt.xlabel('Graduation Year')
plt.ylabel('Impact Score')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Predict future values
future_years_cnn_scaled = scaler.transform(future_years).reshape(-1, 1, 1)
future_cnn_predictions = cnn_model.predict(future_years_cnn_scaled).flatten()

print("\nCNN Forecast for future years:")
for year, prediction in zip(future_years.flatten(), future_cnn_predictions):
    print(f"Year {year}: {prediction:.2f}")

# Update model comparison
models['Improved CNN'] = (mse_cnn, r2_cnn)
print("\nUpdated Model Comparison:")
for model_name, (mse_value, r2_value) in models.items():
    print(f"{model_name} - MSE: {mse_value:.2f}, R-squared: {r2_value:.4f}")

In [ ]:
# Improved Transformer Model for Time Series Prediction
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
import math

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, d_model):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        self.sequence_length = sequence_length
        
    def call(self, x):
        seq_len = tf.shape(x)[1]
        pos = tf.range(seq_len, dtype=tf.float32)[:, tf.newaxis]
        dim = tf.range(self.d_model, dtype=tf.float32)[tf.newaxis, :]
        
        angle_rates = 1 / tf.pow(10000.0, (2 * (dim // 2)) / tf.cast(self.d_model, tf.float32))
        angle_rads = pos * angle_rates
        
        pos_encoding = tf.concat([
            tf.sin(angle_rads[:, 0::2]),
            tf.cos(angle_rads[:, 1::2])
        ], axis=-1)
        
        return x + pos_encoding

class SimpleTransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, dropout=0.1):
        super(SimpleTransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=d_model//num_heads,  # Smaller key dimension
            dropout=dropout
        )
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu", kernel_regularizer=l2(0.01)),
            Dropout(dropout),
            Dense(d_model, kernel_regularizer=l2(0.01)),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)

    def call(self, inputs, training=None):
        # Self-attention with residual connection
        attn_output = self.attention(inputs, inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        
        # Feed forward with residual connection
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Reduce sequence length for small dataset
sequence_length = 3  # Reduced from 5 to get more training samples
d_model = 32  # Reduced from 64 to prevent overfitting

def create_sequences(X, y, seq_length):
    sequences = []
    targets = []
    for i in range(len(X) - seq_length):
        sequences.append(X[i:i+seq_length])
        targets.append(y[i+seq_length])
    return np.array(sequences), np.array(targets)

# Create sequences with additional features
X_seq, y_seq = create_sequences(X_scaled.flatten(), y, sequence_length)

# Add more features: year differences and moving averages
X_enhanced = []
for i in range(len(X_seq)):
    seq = X_seq[i]
    # Calculate differences and moving average
    diffs = np.diff(seq, prepend=seq[0])
    moving_avg = np.convolve(seq, np.ones(len(seq))/len(seq), mode='same')
    
    # Stack features: [original, differences, moving_avg]
    enhanced_seq = np.column_stack([seq, diffs, moving_avg])
    X_enhanced.append(enhanced_seq)

X_seq = np.array(X_enhanced)
print(f"Enhanced sequence shape: {X_seq.shape}")

# Split data with more training samples
train_size = max(1, int(len(X_seq) * 0.85))  # Use 85% for training
X_seq_train, X_seq_test = X_seq[:train_size], X_seq[train_size:]
y_seq_train, y_seq_test = y_seq[:train_size], y_seq[train_size:]

print(f"Training sequences: {len(X_seq_train)}, Test sequences: {len(X_seq_test)}")

# Create a simpler, more appropriate Transformer model
def create_improved_transformer_model(seq_length, feature_dim, d_model=32, num_heads=2, ff_dim=64):
    inputs = Input(shape=(seq_length, feature_dim))
    
    # Smaller embedding projection
    x = Dense(d_model, kernel_regularizer=l2(0.01))(inputs)
    
    # Add positional encoding
    x = PositionalEncoding(seq_length, d_model)(x)
    
    # Single transformer block (reduced complexity)
    x = SimpleTransformerBlock(d_model, num_heads, ff_dim, dropout=0.2)(x)
    
    # Use LSTM as well for sequence understanding
    x = tf.keras.layers.LSTM(32, return_sequences=False, dropout=0.2)(x)
    
    # Simpler output layers
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs, outputs)
    return model

if len(X_seq_train) > 0:
    # Create and compile the improved model
    transformer_model = create_improved_transformer_model(
        sequence_length, 
        X_seq.shape[2],  # Number of features
        d_model=d_model
    )
    
    # Use a lower learning rate and different optimizer
    transformer_model.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0005, weight_decay=0.01),
        loss='mse',
        metrics=['mae']
    )

    print("Improved Transformer Model Architecture:")
    transformer_model.summary()

    # Enhanced callbacks
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, min_delta=1e-4),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6),
        tf.keras.callbacks.ModelCheckpoint('best_transformer.h5', save_best_only=True, monitor='val_loss')
    ]
    
    # Train with reduced epochs and smaller batch size
    history_transformer = transformer_model.fit(
        X_seq_train, y_seq_train,
        epochs=100,  # Reduced from 500
        batch_size=2,  # Smaller batch size
        validation_split=0.15,  # Smaller validation split
        callbacks=callbacks,
        verbose=1
    )
    
    # Make predictions
    y_transformer_pred = transformer_model.predict(X_seq_test).flatten()
    
    # Evaluate the model
    mse_transformer = mean_squared_error(y_seq_test, y_transformer_pred)
    r2_transformer = r2_score(y_seq_test, y_transformer_pred)
    print(f'Improved Transformer Mean Squared Error: {mse_transformer:.2f}')
    print(f'Improved Transformer R-squared: {r2_transformer:.4f}')
    
    # Plot results with better visualization
    plt.figure(figsize=(15, 5))
    
    # Plot 1: Model predictions vs actual
    plt.subplot(1, 3, 1)
    plt.bar(years, y, color='blue', alpha=0.5, label='Actual', width=0.8)
    
    # Get corresponding years for test predictions
    test_years = years[sequence_length + train_size:]
    if len(test_years) == len(y_transformer_pred):
        plt.scatter(test_years, y_transformer_pred, color='red',
                   s=100, label='Transformer Predictions', zorder=5, marker='D')
    
    plt.title('Transformer: Predictions vs Actual')
    plt.xlabel('Graduation Year')
    plt.ylabel('Impact Score')
    plt.legend()
    plt.xticks(rotation=45)
    
    # Plot 2: Prediction accuracy
    plt.subplot(1, 3, 2)
    if len(y_seq_test) > 0:
        plt.scatter(y_seq_test, y_transformer_pred, alpha=0.7, color='green')
        plt.plot([y_seq_test.min(), y_seq_test.max()], 
                 [y_seq_test.min(), y_seq_test.max()], 'r--', lw=2)
        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plt.title('Prediction Accuracy')
        plt.grid(True, alpha=0.3)
    
    # Plot 3: Training history
    plt.subplot(1, 3, 3)
    plt.plot(history_transformer.history['loss'], label='Training Loss', linewidth=2)
    if 'val_loss' in history_transformer.history:
        plt.plot(history_transformer.history['val_loss'], label='Validation Loss', linewidth=2)
    plt.title('Training History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.yscale('log')
    
    plt.tight_layout()
    plt.show()
    
    # Predict future values with the enhanced features
    if len(X_scaled) >= sequence_length:
        last_sequence_simple = X_scaled[-sequence_length:].flatten()
        
        future_transformer_predictions = []
        current_sequence = last_sequence_simple.copy()
        
        for i in range(5):  # Predict 5 years ahead
            # Create enhanced features for prediction
            diffs = np.diff(current_sequence, prepend=current_sequence[0])
            moving_avg = np.convolve(current_sequence, np.ones(len(current_sequence))/len(current_sequence), mode='same')
            enhanced_seq = np.column_stack([current_sequence, diffs, moving_avg])
            enhanced_seq = enhanced_seq.reshape(1, sequence_length, 3)
            
            next_pred = transformer_model.predict(enhanced_seq, verbose=0)[0, 0]
            future_transformer_predictions.append(next_pred)
            
            # Update sequence: remove first, add predicted (scaled)
            next_year_scaled = scaler.transform([[years.max() + i + 1]])[0, 0]
            current_sequence = np.roll(current_sequence, -1)
            current_sequence[-1] = next_year_scaled
    
        print("\nImproved Transformer Forecast:")
        for i, prediction in enumerate(future_transformer_predictions):
            print(f"Year {2022 + i}: {prediction:.2f}")
    
        # Add to model comparison
        models['Improved Transformer'] = (mse_transformer, r2_transformer)
        
        # Final model comparison
        print("\nFinal Model Comparison:")
        print("-" * 60)
        sorted_models = sorted(models.items(), key=lambda x: x[1][0])  # Sort by MSE
        for model_name, (mse_value, r2_value) in sorted_models:
            print(f"{model_name:20} - MSE: {mse_value:10.2f}, R²: {r2_value:8.4f}")
    
else:
    print("Not enough data to create sequences for Transformer model")
    
    # Alternative: Simple LSTM model for small datasets
    print("\nTrying simpler LSTM approach instead...")
    
    # Create simple sequences for LSTM
    X_simple, y_simple = create_sequences(X_scaled.flatten(), y, 2)  # Even shorter sequences
    X_simple = X_simple.reshape(X_simple.shape[0], X_simple.shape[1], 1)
    
    if len(X_simple) > 5:
        train_size_simple = max(1, int(len(X_simple) * 0.8))
        X_train_simple = X_simple[:train_size_simple]
        X_test_simple = X_simple[train_size_simple:]
        y_train_simple = y_simple[:train_size_simple]
        y_test_simple = y_simple[train_size_simple:]
        
        # Simple LSTM model
        simple_model = tf.keras.Sequential([
            tf.keras.layers.LSTM(16, return_sequences=False, dropout=0.2),
            tf.keras.layers.Dense(8, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1)
        ])
        
        simple_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        simple_model.fit(X_train_simple, y_train_simple, epochs=50, verbose=0)
        
        y_simple_pred = simple_model.predict(X_test_simple).flatten()
        mse_simple = mean_squared_error(y_test_simple, y_simple_pred)
        r2_simple = r2_score(y_test_simple, y_simple_pred)
        
        print(f"Simple LSTM MSE: {mse_simple:.2f}, R²: {r2_simple:.4f}")
        models['Simple LSTM'] = (mse_simple, r2_simple)

In [ ]:
# Step 1: Linear trend as base
from sklearn.linear_model import LinearRegression
base_model = LinearRegression().fit(X_train, y_train)
y_base = base_model.predict(X_train)

# Step 2: Model residuals with SVR
residuals = y_train - y_base
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# Scale residuals for SVR
scaler_y = StandardScaler()
scaled_residuals = scaler_y.fit_transform(residuals.reshape(-1, 1)).ravel()

# Train SVR on residuals
svr_residual = SVR(kernel='rbf', C=100, epsilon=0.1)
svr_residual.fit(X_train, scaled_residuals)

# Predict residuals
residual_pred = scaler_y.inverse_transform(
    svr_residual.predict(X_test).reshape(-1, 1)
).ravel()

def get_predictions(x):
    """Get predictions from the base model and residuals."""
    base_pred = base_model.predict(x)
    residuals_pred = scaler_y.inverse_transform(
        svr_residual.predict(x).reshape(-1, 1)
    ).ravel()
    return base_pred + residuals_pred

# Step 3: Combined prediction
y_pred_hybrid = get_predictions(X_test)

# Evaluate the hybrid model
mse_hybrid = mean_squared_error(y_test, y_pred_hybrid)
r2_hybrid = r2_score(y_test, y_pred_hybrid) 
print(f'Hybrid Model Mean Squared Error: {mse_hybrid}')
print(f'Hybrid Model R-squared: {r2_hybrid}')

# Plot the hybrid model results
plt.figure(figsize=(10, 6))
# Plot actual data as bars using year values
plt.bar(years, y, color='blue', alpha=0.5, label='Actual')
# Get corresponding years for test predictions
X_test_years = scaler.inverse_transform(X_test)
plt.scatter(X_test_years.flatten(), y_pred_hybrid, color='purple',
            s=50, label='Hybrid Predictions', zorder=5)

# Plot hybrid line
X_train_years = scaler.inverse_transform(X_train)
plt.plot(X_train_years.flatten(), get_predictions(X_train), color='orange', linewidth=2, label='Base Model Line')
plt.title('Hybrid Model Impact Score vs Graduation Year')
plt.xlabel('Graduation Year')
plt.ylabel('Impact Score')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
# Add hybrid model to comparison
models['Hybrid Model'] = (mse_hybrid, r2_hybrid)
# Final model comparison
print("\nFinal Model Comparison with Hybrid Model:")
sorted_models = sorted(models.items(), key=lambda x: x[1][0])  # Sort by MSE
for model_name, (mse_value, r2_value) in sorted_models:
    print(f"{model_name:20} - MSE: {mse_value:10.2f}, R²: {r2_value:8.4f}")

# Forecast future values with the hybrid model
import numpy as np
last_year = years.max()  # Find most recent year in data
future_years = np.arange(last_year + 1, last_year + 26).reshape(-1, 1)

# Scale future years using the SAME scaler used for training
future_years_scaled = scaler.transform(future_years)

# Hybrid model prediction
future_linear = base_model.predict(future_years_scaled)
future_residuals = svr_residual.predict(future_years_scaled)
# future_predictions = future_linear + future_residuals
future_predictions = get_predictions(future_years_scaled)

plt.figure(figsize=(12, 7))

# Historical actual data
plt.bar(years, y, color='blue', alpha=0.5, label='Historical Actual')

# Historical predictions (from previous plot)
plt.scatter(X_test_years.flatten(), y_pred_hybrid, color='purple', 
            s=50, label='Hybrid Predictions', zorder=5)

# Future forecast
plt.scatter(future_years, future_predictions, color='red', s=100, 
           marker='X', label='Forecast', zorder=10)

# Trend line extension
extended_years = np.vstack([X_train, future_years_scaled])
extended_hybrid = get_predictions(extended_years)

# Inverse scale years for plotting
extended_years_orig = scaler.inverse_transform(extended_years)
plt.plot(extended_years_orig, extended_hybrid, 'g--', linewidth=1.5, 
         label='Forecast Trend')

plt.title('Impact Score Forecast (2025-2029)')
plt.xlabel('Graduation Year')
plt.ylabel('Impact Score')
plt.legend()
plt.grid(alpha=0.3)
plt.xticks(np.arange(1950, 2030, 10))
plt.tight_layout()
plt.show()